<a href="https://colab.research.google.com/github/SamiCakiral/escrim-stock-management/blob/main/R%26D_IMT_Mines_Al%C3%A8s_YOLO_XSENS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install super-gradients

In [ ]:
import torch
import os
import pathlib
import re
from imutils import paths
from IPython.display import YouTubeVideo

# Instantiate the model!

Start by instantiating a pretrained model. YOLO-NAS-Pose comes in three flavors: `yolo_nas_pose_s`, `yolo_nas_pose_m`, and `yolo_nas_pose_l`.

You'll use `yolo_nas_pose_l` throughout this notebook. Because you should always go big, or go home.

It's a good life philosophy.

**Note:** I am using a High-RAM instance of a T4 when running this notebook. If you don't have access to the High-RAM instance, I suggest trying the medium or small versions of the model. If you think you have enough RAM, but find this taking too much memory for your liking, set `fuse_model=False` in the `predict` method.

In [ ]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

yolo_nas_pose = models.get("yolo_nas_pose_l", pretrained_weights="coco_pose").cuda()

[2024-04-22 19:45:55] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-04-22 19:46:07] INFO - utils.py - NumExpr defaulting to 2 threads.
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: torchmetrics required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: hydra-core required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py - Failed to verify installed packages: omegaconf required but not found
[2024-04-22 19:46:11] WARNING - env_sanity_check.py 

## 🖼️  Predicting is a one-liner: `yolo_nas_pose.to(device).predict(input_file)`

Once the model has been instantiated all you have to do is call the `predict` method. The `predict()` method is built to handle multiple data formats and types.

Here is the full list of what `predict()` method can handle:

| Argument Semantics                 | Argument Type      | Supported layout                  | Example                                                                                        | Notes                                                                                            |
|------------------------------------|--------------------|-----------------------------------|------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| Path to local image                | `str`              | -                                 | `predict("path/to/image.jpg")`                                                                 | All common image extensions are supported.                                                       |
| Path to images directory           | `str`              | -                                 | `predict("path/to/images/directory")`                                                          |                                                                                                  |
| Path to local video                | `str`              | -                                 | `predict("path/to/video.mp4")`                                                                 | All common video extensions are supported.                                                       |
| URL to remote image                | `str`              | -                                 | `predict("https://example.com/image.jpg")`                                                     |                                                                                                  |
| 3-dimensional Numpy image          | `np.ndarray`       | `[H, W, C]`                       | `predict(np.zeros((480, 640, 3), dtype=np.uint8))`                                             | Channels last, RGB channel order for 3-channel images                                            |
| 4-dimensional Numpy image          | `np.ndarray`       | `[N, H, W, C]` or `[N, C, H, W]`  | `predict(np.zeros((480, 640, 3), dtype=np.uint8))`                                             | Tensor layout (NHWC or NCHW) is inferred w.r.t to number of input channels of underlying model   |
| List of 3-dimensional numpy arrays | `List[np.ndarray]` | `[H1, W1, C]`, `[H2, W2, C]`, ... | `predict([np.zeros((480, 640, 3), dtype=np.uint8), np.zeros((384, 512, 3), dtype=np.uint8) ])` | Images may vary in size, but should have same number of channels                                 |
| 3-dimensional Torch Tensor         | `torch.Tensor`     | `[H, W, C]` or `[C, H, W]`        | `predict(torch.zeros((480, 640, 3), dtype=torch.uint8))`                                       | Tensor layout (HWC or CHW) is inferred w.r.t to number of input channels of underlying model     |
| 4-dimensional Torch Tensor         | `torch.Tensor`     | `[N, H, W, C]` or `[N, C, H, W]`  | `predict(torch.zeros((4, 480, 640, 3), dtype=torch.uint8))`                                    | Tensor layout (NHWC or NCHW) is inferred w.r.t to number of input channels of underlying model   |

**Important note** - When using batched input (4-dimensional `np.ndarray` or `torch.Tensor`) formats, **normalization and size preprocessing will be applied to these inputs**.




### For more details about predicting using YOLO-NAS-Pose, check out the [docs](https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/ModelPredictions.md)

### Let's wrap the predict method in a function just for this notebook. But really it's just one line to predict! And this is it: `yolo_nas_pose.to(device).predict(input_file)`

In [ ]:
# @title Run this cell for prediction function
def make_prediction(input_file, action, confidence=0.55):
    """
    Make a prediction using the fixed model and device, and either show or save the result.

    Args:
    - input_file (str): Path to the input file.
    - action (str): Either 'show' or 'save'.
    - confidence (float, optional): Confidence threshold. Defaults to 0.75.

    Returns:
    - None

    Raises:
    - ValueError: If the action is not 'show' or 'save'.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    if action == "show":
        yolo_nas_pose.to(device).predict(input_file, conf=confidence).show()
    elif action == "save":
        output_file = pathlib.Path(input_file).stem + "-detections" + pathlib.Path(input_file).suffix
        yolo_nas_pose.to(device).predict(input_file, conf=confidence).save(output_file)
        print(f"Prediction saved to {output_file}")
    else:
        raise ValueError("Action must be either 'show' or 'save'.")

# 📽️ Inference on video

# Load Videos

Code pour charger toute les vidéos de test

In [ ]:
import gdown
import zipfile
import os

# URL du fichier ZIP sur Google Drive
url = 'https://drive.google.com/uc?id=1-k23dMqskWcYF4KAoMA0yql5kO-1LQRk'
https://drive.google.com/file/d/18stBvPNgiNPPYHZGjuaQq2ym7UZ798Fo/view?usp=share_link
output_zip_path = '/content/video_archive.zip'

# Dossier cible pour extraire le contenu du fichier ZIP
extract_to_folder = '/content/video'

# Créez le dossier s'il n'existe pas
if not os.path.exists(extract_to_folder):
    os.makedirs(extract_to_folder)

# Téléchargez le fichier ZIP
gdown.download(url, output_zip_path, quiet=False)

# Décompressez le fichier ZIP
with zipfile.ZipFile(output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

# Supprimez le fichier ZIP si vous n'en avez plus besoin
os.remove(output_zip_path)

print(f'Les vidéos ont été extraites dans {extract_to_folder}')

url ='https://drive.google.com/uc?id=1hUFaYw6RR2couxLqBJ1tQm0mp_1_Qj-k'
output = 'xsensData.xlsx'  # Remplacez par le nom que vous souhaitez donner au fichier téléchargé

gdown.download(url, output, quiet=False)



Downloading...
From (original): https://drive.google.com/uc?id=1-k23dMqskWcYF4KAoMA0yql5kO-1LQRk
From (redirected): https://drive.google.com/uc?id=1-k23dMqskWcYF4KAoMA0yql5kO-1LQRk&confirm=t&uuid=476af17c-6c74-437f-a66e-c1dffe0b5c5e
To: /content/video_archive.zip
100%|██████████| 1.91G/1.91G [00:21<00:00, 88.2MB/s]


Les vidéos ont été extraites dans /content/video


Downloading...
From: https://drive.google.com/uc?id=1hUFaYw6RR2couxLqBJ1tQm0mp_1_Qj-k
To: /content/xsensData.xlsx
  0%|          | 0.00/44.1M [00:00<?, ?B/s]

'xsensData.xlsx'

In [ ]:
import gdown
import zipfile
import os

# URL du fichier ZIP sur Google Drive
url = 'https://drive.google.com/uc?id=18stBvPNgiNPPYHZGjuaQq2ym7UZ798Fo'

output_zip_path = '/content/csv.zip'

# Dossier cible pour extraire le contenu du fichier ZIP
extract_to_folder = '/content/csv'

# Créez le dossier s'il n'existe pas
if not os.path.exists(extract_to_folder):
    os.makedirs(extract_to_folder)

# Téléchargez le fichier ZIP
gdown.download(url, output_zip_path, quiet=False)

# Décompressez le fichier ZIP
with zipfile.ZipFile(output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

# Supprimez le fichier ZIP si vous n'en avez plus besoin
os.remove(output_zip_path)

print(f'Les fichiers CSV ont été extraits dans {extract_to_folder}')



Downloading...
From (original): https://drive.google.com/uc?id=1-k23dMqskWcYF4KAoMA0yql5kO-1LQRk
From (redirected): https://drive.google.com/uc?id=1-k23dMqskWcYF4KAoMA0yql5kO-1LQRk&confirm=t&uuid=476af17c-6c74-437f-a66e-c1dffe0b5c5e
To: /content/video_archive.zip
100%|██████████| 1.91G/1.91G [00:21<00:00, 88.2MB/s]


Les vidéos ont été extraites dans /content/video


Downloading...
From: https://drive.google.com/uc?id=1hUFaYw6RR2couxLqBJ1tQm0mp_1_Qj-k
To: /content/xsensData.xlsx
  0%|          | 0.00/44.1M [00:00<?, ?B/s]

'xsensData.xlsx'

# Fonction pour le calcul des angles et du score + Squelette

Pour analyser les poses humaines à partir de vidéos ou d'images en utilisant le modèle **YoloNAS**, qui est entraîné sur le dataset **COCO**, nous avons développé un protocole spécifique pour extraire, calculer et analyser les angles formés par différents points du squelette humain. Voici une explication claire du processus en plusieurs étapes :

### 1. **Extraction des Keypoints**

Chaque prédiction de pose fournie par YoloNAS renferme un ensemble de points clés (**keypoints**) correspondant aux articulations du corps humain, disposés selon la norme COCO. Ces keypoints incluent des parties cruciales du corps telles que les épaules, les coudes, les poignets, les hanches, les genoux et les chevilles, ainsi que le nez, les yeux et les oreilles, offrant une représentation complète du squelette humain.

### 2. **Définition des Angles d'Intérêt**

Pour procéder à l'analyse, nous définissons d'abord les angles d'intérêt en fonction des connexions naturelles entre les articulations. Par exemple, l'angle du coude gauche est formé par les keypoints de l'épaule gauche, du coude gauche et du poignet gauche. Cette approche est appliquée à diverses parties du corps pour extraire des informations pertinentes sur la posture.

### 3. **Calcul des Angles**

La fonction `calculate_angle` calcule l'angle en degrés au point B (l'articulation centrale, par exemple, le coude ou le genou) en utilisant la loi des cosinus. Cette loi mathématique permet de déterminer l'angle formé entre deux segments de ligne (AB et BC) à partir des longueurs des côtés d'un triangle et du produit scalaire des vecteurs. Ce calcul est essentiel pour quantifier la position et le mouvement des différentes parties du corps.

### 4. **Extraction et Préparation des Données**

Nous avons développé la fonction `extract_keypoints_for_angles` pour extraire spécifiquement les coordonnées nécessaires au calcul des angles d'intérêt à partir des prédictions de pose. Cette fonction filtre les poses en fonction d'un seuil de confiance pour assurer la fiabilité des données extraites.

### 5. **Stockage et Analyse des Résultats**

Les angles calculés pour chaque frame d'une vidéo ou pour chaque image sont ensuite stockés dans un DataFrame pandas par la fonction `calculate_and_store_angles`. Ce DataFrame organise les données de manière à faciliter l'analyse, avec une ligne par frame et une colonne pour chaque angle calculé, permettant ainsi une visualisation claire de l'évolution des postures dans le temps.

### Résumé

En somme, ce protocole offre une méthode systématique et précise pour l'analyse des postures humaines en se basant sur les prédictions de pose du modèle **YoloNAS** entraîné sur le dataset **COCO**. Grâce à l'extraction minutieuse des keypoints, au calcul rigoureux des angles entre les articulations et à l'organisation structurée des données résultantes, nous pouvons obtenir des insights détaillés sur la posture et le mouvement des sujets capturés dans les vidéos ou les images analysées.


In [ ]:
# @title Fonctions pour le calcul d'angle

import numpy as np

def calculate_angle(A, B, C):
    """
    Calcule l'angle en degrés au point B entre les segments formés par les points A, B, et C.

    Args:
    - A, B, C: Coordonnées des points (x, y) sous forme de tuples ou listes.

    Returns:
    - angle_degrees: L'angle en degrés au point B.
    """
    BA = np.array(A) - np.array(B)
    BC = np.array(C) - np.array(B)
    cosine_angle = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    angle = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle)
    return angle_degrees

def extract_keypoints_for_angles(image_prediction, confidence=0.55):
    """
    Extrait les coordonnées des keypoints pour les poses détectées dans une image,
    filtrées par un seuil de confiance, pour une structure de pose basée sur COCO.

    Args:
    - image_prediction (ImagePoseEstimationPrediction): Le résultat de la prédiction pour une image unique.
    - confidence (float, optional): Seuil de confiance pour filtrer les poses. Par défaut à 0.55.

    Returns:
    - dict: Un dictionnaire contenant les coordonnées des keypoints pour les angles d'intérêt.
    """
    # Définition basée sur COCO keypoints
    skeleton_structure = {
        'Bras Gauche': (11, 5, 9),  # Hanche gauche, Épaule gauche, Poignet gauche
        'Bras Droit': (12, 6, 10),  # Hanche droite, Épaule droite, Poignet droit
        'Coude Gauche': (5, 7, 9),  # Épaule gauche, Coude gauche, Poignet gauche
        'Coude Droit': (6, 8, 10),  # Épaule droite, Coude droit, Poignet droit
        'Jambe Gauche': (0, 11, 15),  # Nez (comme substitut pour le buste), Hanche gauche, Cheville gauche
        'Jambe Droite': (0, 12, 16),  # Nez (comme substitut pour le buste), Hanche droite, Cheville droite
        'Genou Gauche': (11, 13, 15),  # Hanche gauche, Genou gauche, Cheville gauche
        'Genou Droit': (12, 14, 16),  # Hanche droite, Genou droit, Cheville droite
        'Poignet Gauche': (5, 7, 9),  # Épaule gauche, Coude gauche, Poignet gauche
        'Poignet Droit': (6, 8, 10)   # Épaule droite, Coude droit, Poignet droit
    }


    keypoints_coordinates = {}
    if image_prediction.prediction.scores.size > 0 and image_prediction.prediction.poses.size > 0:
      # Assurez-vous d'extraire les coordonnées pour la pose la plus confiante
      if image_prediction.prediction.scores[0] > confidence:
          pose = image_prediction.prediction.poses[0]  # Prendre la première pose pour l'exemple

          for angle_name, (p1, p2, p3) in skeleton_structure.items():
              # Extraire les coordonnées (x, y) pour chaque groupe de points
              A = (pose[p1][0], pose[p1][1])
              B = (pose[p2][0], pose[p2][1])
              C = (pose[p3][0], pose[p3][1])

              keypoints_coordinates[angle_name] = [A, B, C]

    return keypoints_coordinates


import pandas as pd



def calculate_and_store_angles(image_predictions, confidence=0.55):
    """
    Pour chaque prédiction d'image, calcule les angles spécifiés et stocke les résultats dans un DataFrame
    avec une ligne par frame et une colonne pour chaque angle.

    Args:
    - image_predictions (List[ImagePoseEstimationPrediction]): Liste des prédictions pour chaque image/frame.
    - confidence (float): Seuil de confiance pour prendre en compte une prédiction.

    Returns:
    - DataFrame: Un DataFrame contenant les angles calculés pour chaque frame/image, structuré par frame.
    """
    # Initialisation d'une liste pour stocker les données de chaque frame/image
    angles_data = []

    for i, image_prediction in enumerate(image_predictions):
        # Dictionnaire pour stocker les angles de la frame courante
        frame_angles = {'Frame': i}

        # Extraction des coordonnées des keypoints pour les angles d'intérêt
        keypoints_coordinates = extract_keypoints_for_angles(image_prediction, confidence)

        # Calcul des angles pour chaque groupe de keypoints d'intérêt
        for angle_name, coords_group in keypoints_coordinates.items():
            # Assurez-vous que nous avons exactement trois points (A, B, C)
            if len(coords_group) == 3:
                A, B, C = coords_group
                angle = calculate_angle(A, B, C)
                # Ajoutez l'angle calculé au dictionnaire de la frame courante
                frame_angles[f'Angle {angle_name}'] = angle

        # Ajoutez les angles de la frame courante à la liste des données
        angles_data.append(frame_angles)

    # Créez le DataFrame à partir de la liste des données
    df = pd.DataFrame(angles_data)

    # Réordonnez les colonnes pour mettre 'Frame' en première position si nécessaire
    cols = ['Frame'] + [col for col in df.columns if col != 'Frame']
    df = df[cols]

    return df


import numpy as np
import cv2
import pathlib

def process_single_image(image_prediction, confidence=0.55):
    """
    Process a single image prediction by drawing the detected poses on the image.

    Args:
    - image_prediction (ImagePoseEstimationPrediction): The prediction result for a single image.
    - confidence (float, optional): Confidence threshold. Defaults to 0.55.

    Returns:
    - np.ndarray: The processed image with drawn poses.
    """
    # Assurez-vous que le seuil de confiance est respecté pour chaque prédiction
    filtered_poses = [pose for pose, score in zip(image_prediction.prediction.poses, image_prediction.prediction.scores) if score > confidence]

    # Dessinez les poses filtrées sur l'image. Vous pouvez adapter les arguments en fonction de vos besoins spécifiques
    processed_image = image_prediction.draw(
        joint_thickness=2,  # Épaisseur des joints
        keypoint_radius=3,  # Rayon des keypoints
        box_thickness=2,    # Épaisseur des boîtes
        show_confidence=True  # Afficher les scores de confiance
    )

    return processed_image

import cv2

def create_video_from_frames(frames, output_path, fps):
    """
    Crée une vidéo à partir d'une liste de frames et la sauvegarde au chemin spécifié.

    Args:
    - frames (List[np.ndarray]): Liste des images (frames) à assembler en vidéo.
    - output_path (str): Chemin du fichier de sortie pour la vidéo créée.
    - fps (int): Nombre de frames par seconde pour la vidéo.

    Returns:
    - None
    """

    # Vérifiez qu'il y a des frames à traiter
    if not frames:
        raise ValueError("La liste des frames est vide.")

    # Récupérez la hauteur et la largeur de la première frame
    height, width = frames[0].shape[:2]

    # Définissez le codec et créez l'objet VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour les fichiers .mp4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Écrivez chaque frame dans le fichier de sortie
    for frame in frames:
        out.write(frame)

    # Libérez l'objet VideoWriter
    out.release()



100%|██████████| 44.1M/44.1M [00:00<00:00, 104MB/s]


### Calcul d'angle avec une seule POV

Dans cette étape, nous effectuons la détection de pose et le calcul d'angles pour une **POV (Point Of View)** unique. Nous utilisons le modèle **Yolo-NAS** pour prédire les poses à partir d'une vidéo, puis nous appliquons une fonction pour extraire les **keypoints** et calculer les angles entre certains points d'intérêt sur le squelette détecté. Cette approche nous permet d'obtenir des mesures d'angles spécifiques à partir d'un seul angle de vue.

Le processus est le suivant :
1. **Prédiction des poses** : Utilisation de Yolo-NAS pour détecter les poses dans chaque frame de la vidéo.
2. **Extraction des keypoints** : Sélection des points d'intérêt pour le calcul d'angles en se basant sur la structure du squelette COCO.
3. **Calcul des angles** : Application d'une formule géométrique pour déterminer l'angle entre trois points choisis, offrant ainsi une mesure de l'angle d'intérêt pour chaque frame.

**Importance** : Cette cellule nous fournit une base solide pour comprendre comment les angles sont calculés à partir des données de pose estimées par le modèle pour une seule perspective. Cela est essentiel pour les étapes suivantes où nous envisageons d'intégrer des données de multiples POV.


In [ ]:
# @title Calcul d'angle avec une seule POV

import pandas as pd

# Initialisez une liste pour stocker les données des angles pour chaque frame
angles_data = []
processed_frames = []
result = yolo_nas_pose.to('cuda').predict("video/front1.avi", conf=.4)
# Remplacez ceci par votre boucle existante pour traiter chaque frame
for frame_index, image_prediction in enumerate(result._images_prediction_gen):
    # Ici, nous extrayons les keypoints et calculons les angles au lieu de simplement traiter l'image
    keypoints_coordinates = extract_keypoints_for_angles(image_prediction, confidence=0.55)

    # Calcul des angles pour chaque groupe de keypoints d'intérêt
    for angle_name, coords_group in keypoints_coordinates.items():
        # Assurez-vous que nous avons exactement trois points (A, B, C)
        if len(coords_group) == 3:
            A, B, C = coords_group
            angle = calculate_angle(A, B, C)
            # Ajoutez le résultat dans la liste des données
            angles_data.append({'Frame': frame_index, 'Angle Name': angle_name, 'Angle': angle})

# Créez le DataFrame à partir de la liste des données
df_angles = pd.DataFrame(angles_data)



[2024-04-22 16:29:48] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
<ipython-input-6-61f1e69385e9>:18: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)


### Calcul d'angle avec toutes les pov superposées (Useless mais codé)

Cette cellule explore l'idée de superposer les données de pose de **multiples POV** et de sélectionner la meilleure frame basée sur le **score de confiance** pour chaque instant. Bien que théoriquement intéressante, cette approche s'est avérée peu pratique en raison des complications liées à la fusion des perspectives et à l'alignement des données dans un espace tridimensionnel complexe.

**Raisonnement** :
- **Superposition des données** : Tentative de combiner les informations de pose à partir de différentes angles de vue en une représentation unifiée.
- **Sélection par score de confiance** : Choix de la frame avec le meilleur score de confiance pour chaque instant, sans tenir compte de la cohérence spatiale entre les différentes POV.

**Conclusion** : Bien que réalisable, cette méthode introduit une complexité significative sans garantir une amélioration de la précision ou de la pertinence des mesures d'angle. Elle a donc été jugée moins utile pour notre objectif de mesure précise et cohérente des angles à travers différentes POV.


In [ ]:
# @title Calcul d'angle avec toute les pov superposées (Useless mais codé)

import os

def process_and_select_best_frames_from_videos(video_folder, confidence_threshold=0.55):
    video_files = [file for file in os.listdir(video_folder) if file.endswith('.mp4')]
    best_frames_info = []  # Pour stocker le meilleur frame et son score pour chaque frame_index

    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        result = yolo_nas_pose.to('cuda').predict(video_path, conf=confidence_threshold)

        for frame_index, image_prediction in enumerate(result._images_prediction_gen):
            # Obtenez les keypoints et calculez les angles pour la frame actuelle
            keypoints_coordinates = extract_keypoints_for_angles(image_prediction, confidence=confidence_threshold)

            # Obtenez le score de confiance global de la frame (supposant une logique pour l'obtenir)
            frame_confidence_score = image_prediction.prediction.scores[0]  # Exemple hypothétique

            if len(best_frames_info) <= frame_index:
                # Si nous n'avons pas encore traité ce frame_index dans d'autres vidéos
                best_frames_info.append({'frame': image_prediction, 'score': frame_confidence_score})
            elif frame_confidence_score > best_frames_info[frame_index]['score']:
                # Si le score de confiance actuel est meilleur que le meilleur précédent
                best_frames_info[frame_index] = {'frame': image_prediction, 'score': frame_confidence_score}

    # Après avoir examiné toutes les vidéos, dessinez les squelettes pour les meilleures frames sélectionnées
    processed_frames = [process_single_image(frame_info['frame']) for frame_info in best_frames_info]

    # Créez une vidéo à partir des meilleures frames sélectionnées
    #create_video_from_frames(processed_frames, 'best_poses_video.mp4', fps=30)  # Assumer fps=30 pour l'exemple

# Appliquez cette fonction au dossier contenant vos vidéos
video_folder = "video/zipedVideo"
process_and_select_best_frames_from_videos(video_folder)


[2024-04-17 15:54:42] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


### Calcul d'angle moyen pondéré par le score de confiance avec toutes les pov

Dans cette cellule, nous adoptons une approche raffinée pour intégrer les données de **multiples POV** en calculant un **angle moyen pondéré** basé sur les **scores de confiance**. Cette méthode tire parti des forces de chaque POV individuelle tout en atténuant les faiblesses potentielles liées à des angles de vue spécifiques.

**Procédure** :
1. **Calcul individuel des angles** : Pour chaque POV, nous calculons les angles et les scores de confiance associés.
2. **Moyenne pondérée** : Les angles de chaque frame sont ensuite combinés en utilisant une moyenne pondérée par les scores de confiance, ce qui permet de privilégier les données les plus fiables.

**Avantages** :
- **Exploitation optimale des POV multiples** : Cette méthode permet de bénéficier de la diversité des points de vue sans les contraintes liées à leur superposition directe.
- **Amélioration de la précision** : La moyenne pondérée augmente la fiabilité des mesures d'angle en donnant plus de poids aux estimations les plus confiantes.

**Impact** : En fin de compte, cette approche nous offre une méthode robuste pour analyser les mouvements et postures à partir de vidéos capturées à partir de multiples angles, en assurant une intégration intelligente et efficace des données.


In [ ]:
# @title Calcul d'angle moyen pondéré par confidance score avec toute les pov

import pandas as pd
import numpy as np

def calculate_weighted_average_angles(video_folder, confidence_threshold=0.55):
    video_files = [file for file in os.listdir(video_folder) if file.endswith('.avi')]
    all_frames_data = []

    # Étape 1: Calculer les angles pour chaque POV
    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        result = yolo_nas_pose.to('cuda').predict(video_path, conf=confidence_threshold)
        frames_data = []

        for frame_index, image_prediction in enumerate(result._images_prediction_gen):
            keypoints_coordinates = extract_keypoints_for_angles(image_prediction, confidence=confidence_threshold)
            frame_data = {'Frame': frame_index}

            for angle_name, coords_group in keypoints_coordinates.items():
                if len(coords_group) == 3:
                    A, B, C = coords_group
                    angle = calculate_angle(A, B, C)
                    frame_data[f'Angle {angle_name}'] = angle
                    frame_data[f'Confidence {angle_name}'] = np.mean(image_prediction.prediction.scores)  # Exemple de calcul de confiance

            frames_data.append(frame_data)

        # Convertir en DataFrame et stocker dans la liste globale
        df_video = pd.DataFrame(frames_data)
        df_video['POV'] = video_file
        all_frames_data.append(df_video)

    # Étape 2: Fusionner tous les DataFrames de vidéos
    df_all_videos = pd.concat(all_frames_data, ignore_index=True)

    # Calculer la moyenne pondérée pour chaque frame et angle à travers toutes les vidéos
    weighted_average_df = df_all_videos.groupby(['Frame', 'POV']).mean().reset_index()

    return df_all_videos, weighted_average_df

# Exemple d'utilisation
video_folder = "video/2"
df_weighted_angles,df_angles_all_videos = calculate_weighted_average_angles(video_folder)

# Affichage des premières lignes du DataFrame final
print(df_weighted_angles.head())


[2024-04-22 17:16:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-04-22 17:18:33] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
<ipython-input-15-1abc26a9f1cb>:18: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
[2024-04-22 17:19:25] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


In [ ]:
df_angles.head()
df_angles.to_csv("anglesFront1.csv")
df_angles_all_videos.to_csv("allVid2Ok.csv")
df_weighted_angles.to_csv('anglesW2Ok.csv')

# Lecture fichier xsens pour sortir un df_angles_xsens

In [ ]:
import pandas as pd
import gdown

def process_xsens_data(xsens_path, sheet_name, column_mapping):


    # Lire les données du fichier Excel xSens
    df_xsens = pd.read_excel(xsens_path, sheet_name=sheet_name)

    # Sélectionner et renommer les colonnes en utilisant le dictionnaire de mappage
    columns_of_interest = list(column_mapping.keys())
    df_xsens_selected = df_xsens[columns_of_interest].rename(columns=column_mapping)

    # Créer un DataFrame avec les angles renommés
    df_angle_xsens = df_xsens_selected.rename(columns=lambda name: 'Angle ' + name)

    return df_angle_xsens


column_mapping = { # Manque des choses, c'est temporaire pour un test
    'Frame': 'Frame',  # Assumer que le numéro de frame est correct
    # Mappage des mesures de l'articulation de l'épaule
    'Left Shoulder Flexion/Extension': 'Bras Gauche Flexion/Extension',
    'Right Shoulder Flexion/Extension': 'Bras Droit Flexion/Extension',
    # Mappage des mesures de l'articulation du coude
    'Left Elbow Flexion/Extension': 'Coude Gauche Flexion/Extension',
    'Right Elbow Flexion/Extension': 'Coude Droit Flexion/Extension',
    # Mappage des mesures de l'articulation de la hanche pour la jambe
    'Left Hip Flexion/Extension': 'Jambe Gauche Flexion/Extension',
    'Right Hip Flexion/Extension': 'Jambe Droite Flexion/Extension',
    # Mappage des mesures de l'articulation du genou
    'Left Knee Flexion/Extension': 'Genou Gauche Flexion/Extension',
    'Right Knee Flexion/Extension': 'Genou Droit Flexion/Extension',
    # Mappage des mesures de l'articulation du poignet
    'Left Wrist Flexion/Extension': 'Poignet Gauche Flexion/Extension',
    'Right Wrist Flexion/Extension': 'Poignet Droit Flexion/Extension',
}


# Utiliser la fonction
xsens_path = 'xsensData.xlsx'
sheet_name = 'Joint Angles ZXY'  # Ou autre feuille du fichier Excel selon vos besoins
df_angle_xsens = process_xsens_data(xsens_path, sheet_name, column_mapping)


In [ ]:
df_angle_xsens

,Angle Frame,Angle Bras Gauche Flexion/Extension,Angle Bras Droit Flexion/Extension,Angle Coude Gauche Flexion/Extension,Angle Coude Droit Flexion/Extension,Angle Jambe Gauche Flexion/Extension,Angle Jambe Droite Flexion/Extension,Angle Genou Gauche Flexion/Extension,Angle Genou Droit Flexion/Extension,Angle Poignet Gauche Flexion/Extension,Angle Poignet Droit Flexion/Extension
0,0,19.834392,-2.398467,13.001976,24.503302,8.355053,5.295085,7.227706,4.974105,7.397145,-0.147721
1,1,19.793270,-2.464427,12.910597,24.615320,8.259817,5.242339,7.174970,4.984867,6.457907,-0.199729
2,2,19.752140,-2.530390,12.819180,24.722739,8.164611,5.189561,7.122247,4.995629,5.519434,-0.255811
3,3,19.709416,-2.585937,12.747469,24.818088,8.077358,5.137920,7.077822,5.005358,4.612105,-0.545159
4,4,19.661376,-2.624198,12.702113,24.894425,8.001356,5.088158,7.045603,5.015176,3.742549,-1.195650
...,...,...,...,...,...,...,...,...,...,...,...
4037,4037,23.637930,5.585755,6.933385,10.380064,4.296421,1.546671,6.051620,3.261265,10.302855,-3.252249
4038,4038,23.621237,5.573018,6.937896,10.386678,4.294296,1.545041,6.049739,3.259248,10.317083,-3.252079
4039,4039,23.600261,5.561873,6.939000,10.381776,4.292516,1.544912,6.050735,3.261254,10.329662,-3.247059
4040,4040,23.576672,5.551231,6.938520,10.370576,4.290861,1.545499,6.053333,3.265330,10.342551,-3.239248


# Fonction pour dessiner le Squelette

In [ ]:
# @title Sur toute les pov
import os

def process_all_videos(video_folder):
    # Liste tous les fichiers dans le dossier
    video_files = [file for file in os.listdir(video_folder) if file.endswith('.mp4')]

    for video_file in video_files:
        # Préparez une liste pour collecter les frames traitées
        processed_frames = []

        # Chemin complet vers la vidéo
        video_path = os.path.join(video_folder, video_file)

        # Prédiction des poses pour la vidéo actuelle
        result = yolo_nas_pose.to('cuda').predict(video_path, conf=.4)

        # Itérer sur chaque prédiction d'image dans le générateur
        for image_prediction in result._images_prediction_gen:
            # Appliquez le traitement nécessaire pour chaque image
            # En utilisant une fonction `process_single_image` pour dessiner les squelettes
            processed_image = process_single_image(image_prediction)
            processed_frames.append(processed_image)

        # Après avoir collecté toutes les frames traitées, créez une vidéo
        # Le nom de la vidéo de sortie est basé sur le nom du fichier d'entrée
        output_video_path = os.path.join(video_folder, f"skeleton_{video_file}")
        create_video_from_frames(processed_frames, output_video_path, fps=result.fps)

# Utilisez cette fonction en spécifiant le chemin du dossier contenant vos vidéos
video_folder = "video/zipedVideo"
process_all_videos(video_folder)


[2024-04-18 18:56:30] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


In [ ]:
import gdown
import zipfile
import os

# URL du fichier ZIP sur Google Drive
url = 'https://drive.google.com/uc?id=1ngVS0Qfn5NqR1yvc64lP8BE-96Dg3Lvi'

output_zip_path = '/content/video_archive.zip'

# Dossier cible pour extraire le contenu du fichier ZIP
extract_to_folder = '/content/video2'

# Créez le dossier s'il n'existe pas
if not os.path.exists(extract_to_folder):
    os.makedirs(extract_to_folder)

# Téléchargez le fichier ZIP
gdown.download(url, output_zip_path, quiet=False)

# Décompressez le fichier ZIP
with zipfile.ZipFile(output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

# Supprimez le fichier ZIP si vous n'en avez plus besoin
os.remove(output_zip_path)


Downloading...
From (original): https://drive.google.com/uc?id=1ngVS0Qfn5NqR1yvc64lP8BE-96Dg3Lvi
From (redirected): https://drive.google.com/uc?id=1ngVS0Qfn5NqR1yvc64lP8BE-96Dg3Lvi&confirm=t&uuid=c543a8cb-8243-4951-a79f-e6445021e75c
To: /content/video_archive.zip
100%|██████████| 42.9M/42.9M [00:00<00:00, 51.6MB/s]


In [ ]:
#@title Sur une seule pov


# Préparez une liste pour collecter les frames traitées

processed_frames = []
result = yolo_nas_pose.to('cuda').predict("video2/side1.mp4", conf=.4)
# Itérez sur chaque prédiction d'image dans le générateur
for image_prediction in result._images_prediction_gen:
    # Appliquez le traitement nécessaire pour chaque image
    # Par exemple, en utilisant une fonction fictive `process_single_image` que vous devez définir ou adapter
    processed_image = process_single_image(image_prediction)
    processed_frames.append(processed_image)


# Après avoir collecté toutes les frames traitées, créez une vidéo
# Assurez-vous d'avoir une fonction `create_video_from_frames` qui prend la liste des frames et le chemin de sortie
create_video_from_frames(processed_frames, 'side1Skeleton.mp4', fps=result.fps)

[2024-04-22 19:58:17] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
